In [5]:
import cracknuts as cn

In [6]:
g1 = cn.cracker_g1('192.168.0.10')
g1.connect(force_update_bin=True, force_write_default_config=True)

In [8]:
import random
import time
from cracknuts.cracker import serial
from cracknuts.acquisition.glitch_acquisition import GlitchDoData, GlitchDoStatus


cmd_set_aes_enc_key = "01 00 00 00 00 00 00 10"
cmd_aes_enc = "01 02 00 00 00 00 00 10"

aes_key = "11 22 33 44 55 66 77 88 99 00 aa bb cc dd ee ff"
aes_data_len = 16

sample_length = 20000

def init(cracker):
    
    cracker.nut_clock_freq('8M')
    cracker.nut_clock_enable()
    cracker.nut_voltage(3.3)
    cracker.nut_voltage_enable()
    cracker.uart_io_enable()
    cracker.osc_sample_clock('48m')
    cracker.osc_sample_length(sample_length)
    # cracker.osc_trigger_source('N')
    cracker.osc_trigger_source('P')
    # cracker.osc_analog_enable('A')
    # cracker.osc_analog_gain('A', 1)
    # cracker.osc_analog_disable('B')
    cracker.osc_analog_gain('B', 10)
    cracker.osc_trigger_level(0)
    cracker.osc_trigger_mode('E')
    cracker.osc_trigger_edge('U')
    cracker.uart_config(
        baudrate=serial.Baudrate.BAUDRATE_115200, 
        bytesize=serial.Bytesize.EIGHTBITS, 
        parity=serial.Parity.PARITY_NONE, 
        stopbits=serial.Stopbits.STOPBITS_ONE)
    g1.glitch_vcc_normal(3.3)
    g1.glitch_gnd_normal(0.0)
    g1.glitch_vcc_config(wait=1, level=3.0, length=10, delay=10, repeat=2)
    g1.glitch_vcc_normal(3.3)
    g1.glitch_gnd_normal(0.0)
    g1.glitch_gnd_config(wait=1, level=0.3, length=10, delay=10, repeat=2)
    g1.glitch_clock_config(
        wave=[1.5, 1.5, 1.5, 1.5, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0],
        wait=1,
        delay=1,
        repeat=1
    )
    time.sleep(2)
    cmd = cmd_set_aes_enc_key + aes_key
    status, ret = cracker.uart_transmit_receive(cmd, timeout=1000, rx_count=6)

def do(cracker, count) -> GlitchDoData:
    # plaintext_data = random.randbytes(aes_data_len)
    # cracker.glitch_clock_arm()
    # cracker.glitch_vcc_arm()
    # cracker.glitch_gnd_arm()
    plaintext_data = bytes.fromhex('AA BB CC DD EE FF 00 11 22 33 44 55 66 77 88 99')
    tx_data = bytes.fromhex(cmd_aes_enc.replace(' ', '')) + plaintext_data
    status, ret = cracker.uart_transmit_receive(tx_data, rx_count= 6 + aes_data_len, is_trigger=True)
    # print(ret.hex(' ') if ret else 'None')
    return {
        "plaintext": plaintext_data,
        "ciphertext": b'',
        "key": bytes.fromhex(aes_key)
    }
    return GlitchDoData(
        plaintext=plaintext_data,
        ciphertext=b'' if ret is None else ret[-6],
        key=bytes.fromhex(aes_key),
        status=GlitchDoStatus.NOT_GLITCHED
    )


acq = cn.simple_glitch_acq(g1, init, do)

p = cn.show_panel(acq)
p

In [ ]:
g1.glitch_vcc_normal(3.3)
g1.glitch_gnd_normal(0.0)
g1.glitch_vcc_config(wait=1, level=2.5, length=10, delay=10, repeat=2)
# g1.nut_voltage_enable()

In [ ]:
g1.glitch_vcc_force()

In [ ]:
g1.glitch_vcc_normal(3.3)
g1.glitch_gnd_normal(0.0)
g1.glitch_gnd_config(wait=1, level=0.5, length=10, delay=10, repeat=2)
# g1.nut_voltage_enable()

In [ ]:
g1.glitch_gnd_force()

In [ ]:
g1.glitch_clock_config(
    wave=[1.5, 1.5, 1.5, 1.5, 1.5, 0.0, 0.0, 0.0, 0.0, 0.0],
    wait=1,
    delay=1,
    repeat=1
)
# g1.nut_clock_enable()
# g1.nut_voltage_enable()

In [ ]:
g1.glitch_clock_force()

In [9]:
import zarr

d = zarr.open(r'dataset/20260108162753.zarr/')
d.info

Name,/
Type,zarr.hierarchy.Group
Read-only,False
Store type,zarr.storage.DirectoryStore
No. members,1
No. arrays,0
No. groups,1
Groups,0


In [11]:
d["/0/0/traces"]

<zarr.core.Array '/0/0/traces' (100, 20000) int16>